In [5]:
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

# Sigmoid example

In [6]:
@tf.function
def layer(x, W, b):
    # building the graph
    return tf.math.sigmoid(tf.matmul(x, W) + b)

In [7]:
x = np.array([[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]], dtype=np.float32)

init_w = tf.initializers.RandomUniform(minval=-.1, maxval=.1)(shape=[10,5])
W = tf.Variable(init_w, dtype=tf.float32, name='W')

init_b = tf.initializers.RandomUniform()(shape=[5])
b = tf.Variable(init_b, dtype=tf.float32, name='b')

h = layer(x, W, b)

In [8]:
print(f"h: {h.numpy()}")

h: [[0.5018517  0.49334908 0.50805944 0.44643217 0.46883595]]


# Defining Inputs
3 different way to feed data to tf program:
- as np array
- as tf tensors
- using `tf.data` **API** to create an input *pipeline*



In [10]:
# feeding data as tensor
x = tf.constant(
    [[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]], 
    dtype=tf.float32,
    name="x"
)

init_w = tf.initializers.RandomUniform(minval=-.1, maxval=.1)(shape=[10,5])
W = tf.Variable(init_w, dtype=tf.float32, name='W')

init_b = tf.initializers.RandomUniform()(shape=[5])
b = tf.Variable(init_b, dtype=tf.float32, name='b')

h = layer(x, W, b)

In [11]:
print(f"h: {h.numpy()}")
print(f"type: {type(h)}")

h: [[0.44630384 0.49760556 0.48088944 0.5210194  0.5187529 ]]
type: <class 'tensorflow.python.framework.ops.EagerTensor'>


In [29]:
# tf.data api
filenames = [f"./iris.data.{i}" for i in range(1,4)]

dataset = tf.data.experimental.CsvDataset(
    filenames,
    [
        tf.float32,
        tf.float32,
        tf.float32,
        tf.float32,
        tf.string,
    ]
)

# organize our data into inputs and labels
dataset = dataset.map(lambda x1,x2,x3,x4,y: (tf.stack([x1,x2,x3,x4]), y))

In [30]:
for next_element in dataset:
    x, y = next_element[0].numpy(), next_element[1].numpy().decode("ascii")
    if np.min(x) < .0:
        print(f"(corrupted) X => {x}\tY => {y}")

(corrupted) X => [ 5.8 -4.   1.2  0.2]	Y => Iris-setosa
(corrupted) X => [ 4.6  3.6 -1.   0.2]	Y => Iris-setosa
(corrupted) X => [ 4.7 -3.2  1.6  0.2]	Y => Iris-setosa
(corrupted) X => [-5.   3.5  1.3  0.3]	Y => Iris-setosa
(corrupted) X => [-5.1  3.8  1.6  0.2]	Y => Iris-setosa
(corrupted) X => [ 6.9 -3.1  4.9  1.5]	Y => Iris-versicolor
(corrupted) X => [-5.2  2.7  3.9  1.4]	Y => Iris-versicolor
(corrupted) X => [ 5.8  2.7  4.1 -1. ]	Y => Iris-versicolor
(corrupted) X => [ 6.1 -2.8  4.7  1.2]	Y => Iris-versicolor
(corrupted) X => [ 5.4  3.  -4.5  1.5]	Y => Iris-versicolor
(corrupted) X => [ 7.1 -3.   5.9  2.1]	Y => Iris-virginica
(corrupted) X => [ 5.7  2.5  5.  -2. ]	Y => Iris-virginica
(corrupted) X => [-6.   2.2  5.   1.5]	Y => Iris-virginica
(corrupted) X => [ 7.2 -3.2  6.   1.8]	Y => Iris-virginica
(corrupted) X => [ 6.1  2.6  5.6 -1.4]	Y => Iris-virginica


In [31]:
dataset = dataset.filter(lambda x, y: tf.reduce_min(x) > 0)

In [32]:
batch_size = 5
dataset = dataset.batch(batch_size=batch_size)
for next_element in dataset:
    x, y = next_element[0].numpy(), next_element[1].numpy()    
    print(f"x.shape = {x.shape}, y.shape = {y.shape}")
    break

x.shape = (5, 4), y.shape = (5,)


In [27]:
x.shape

(4,)